# Investigation of the $a_1(1260)$ pole position
Misha Mikhasenko, HISKP, Universität Bonn

## $a_1(1260)$ parametrization

the amplitude is parametrized by the Breit-Wigner formula with energy-dependent width. The extra factor $m/\sqrt{s}$ is taken from Bowler.
$$
a(s) = \frac{c}{m^2-s-im\Gamma(s)},\qquad
\Gamma(s) = \Gamma_0(s)\frac{\rho(s)}{\rho(m^2)} \frac{m}{\sqrt{s}}
$$


#### Two expressions for $\rho(s)$
 * First, non-symmetrized decay amplitude. The interference between $\rho$-bands is small due to the width of rho. The ampliutude of $a_1(1260)$ is almost uneffected,
 but continuation is way easier.
 * Second, the symmetrized decay amplitude in full glory as used in the compass fit

## Energy dependent width
### Basis recoupling functions

The phase space function $\rho(s)$ is calculated by integrating the decay amplitude squared over the Dalitz Plot
$$
\rho(s) = \frac{1}{2}\int \mathrm{d} \Phi_3 \big[\,f_\rho(s_1) (Z_1)_{Ll}^{JM}(\tau_1) - f_\rho(s_3) (Z_3)_{Ll}^{JM}(\tau_3)\big]^2
$$


To calculate the __phase space integral__ over function which have isobars in both channels
one needs to express all _kinematical variables_ of the cross channel ($s_3,\tau_3$) through the kinematical function of the integrated channel. The function `change of basis` does a couple of boosts and rotations.
$$
s_3,\tau_3 = \text{change_of_basis}(s_1,\tau_1)
$$
 * the dependence between functions is analytic (given by trignometrical functions)

In [3]:
using QuadGK
using Plots
using GSL
using Cuba

In [4]:
push!(LOAD_PATH, ENV["HOME"]*"/Documents/amplitude_analysis/modules")

3-element Array{Any,1}:
 "/localhome/mikhasenko/Tools/julia/usr/local/share/julia/site/v0.6"
 "/localhome/mikhasenko/Tools/julia/usr/share/julia/site/v0.6"      
 "/localhome/mikhasenko/Documents/amplitude_analysis/modules"       

In [5]:
using isobars
using masses
using DalitzPlotAnalysis

### A check if there are discontinuity in the space of the compex parameters.
##### The phase space is determined by 5 variables:
 * isobar invariant mass $s_1$
 * decay angles of isobar in the CMS frame $\cos\theta_1,\phi_1$
 * decay angles of a pion in the isobar helicity frame $\cos\theta_{23},\phi_{23}$

In [36]:
# change_of_basis(s_1,cosθ1,ϕ1,cosθ23,ϕ23,m1sq,m2sq,m3sq,s)
v1,v2 = change_basis(1.0+0.1im,0.3,1.1,0.2,1.3,mπ2,mπ2,mπ2,1.5-0.1im),
        change_basis(1.0+0.1im,0.3,1.1,0.2,1.3,mπ2,mπ2,mπ2,1.5+0.1im)
# print 
vars = ["s_i", "cosθ_i", "ϕ_i", "cosθ_jk", "ϕ_jk"]
println(rpad("variable",20),rpad("s+iϵ, σ-iϵ",20),lpad("s+iϵ, σ+iϵ",20))
println(rpad("",60,"-"))
for i in 1:length(v1)
    println(rpad(vars[i],20),rpad(round(v1[i],2),20),lpad(round(v2[i],2),20))
end

variable            s+iϵ, σ-iϵ                    s+iϵ, σ+iϵ
------------------------------------------------------------
s_i                 0.32 - 0.12im               0.32 - 0.0im
cosθ_i              -0.76 + 0.12im            -0.72 + 0.02im
ϕ_i                 2.1 - 0.38im               2.15 - 0.07im
cosθ_jk             0.74 + 0.19im              0.76 + 0.03im
ϕ_jk                0.92 - 0.28im              0.98 - 0.05im


In [8]:
function second_integral(s)
    function dPhiZ3fZ1sfs(x, f)
        # complex path
        #  end points:
        s1_i = 4mπ2;
        s1_f = (sqrt(s)-mπ)^2 - 1e-8 # otherwise, rounding error
        s1_m = iszero(imag(s1_f)) ? s1_f : real(s1_f)+1e-5im*imag(s1_f);
        #  variables:
        s1 = (x[1] < 0.5) ? s1_i + 2x[1]*(s1_m-s1_i) : s1_m + 2(x[1]-0.4)*(s1_f-s1_m)
        #  jacobian
        res = (x[1] < 0.5) ? 2(s1_m-s1_i) : 2(s1_f-s1_m);
        # angular variables
        τ1 = (2*x[2]-1,2*π*x[3],2*x[4]-1,2*π*x[5])
        τ3 = fill(0.0+0.0im,4);
        s3,τ3[1],τ3[2],τ3[3],τ3[4] = change_basis(s1,τ1[1],τ1[2],τ1[3],τ1[4],mπ2,mπ2,mπ2,s);
        [ if !isfinite(τi)
           print("s = ",s,":\n",τ1,"\n ",τ3,"\n\n")
            end for τi in τ3 ]
        # the angular part of the rho-pi S-wave
        # For the rho pi S-wave, it reads:
        #    cosθ1*cosθ23-sinθ1*sinθ23*cosϕ23
        ang = [
            begin
                cI = τ[1]; sI = sqrt(1.0-cI^2);
                cV = τ[3]; sV = sqrt(1.0-cV^2);
                sqrt(3.)*(cI*cV-sI*sV*cos(τ[4]))
            end for τ in [τ1,τ3] ]
        # value
        amp = f1_I(s1)*ang[1] - f1_I(s3)*ang[2]
        ampc = f1_II(s1)*ang[1] - f1_II(s3)*ang[2]
        res *= amp*ampc/2
        # intagra jacobian factors
        res *= sqrt(λ(s,s1,mπ2)*λ(s1,mπ2,mπ2))/(s*s1)
        f[1],f[2] = reim(res)
    end
    result = cuhre(dPhiZ3fZ1sfs, 5, 2)
    complex(result[1]...) / (2*π) / (8*π)^2
end

second_integral (generic function with 1 method)

### Time for the calculculation of the phase space integral at the complex point

In [10]:
@time second_integral(1.2)

  9.471396 seconds (88.06 M allocations: 2.364 GiB, 5.26% gc time)


0.01717520671645439 + 0.0im

### No discontinuity due to the angular variables

In [12]:
println("s+iϵ...........",round(second_integral(1.4+0.2im),3))
println("s..............",round(second_integral(1.4      ),3))
println("s-iϵ...........",round(second_integral(1.4-0.2im),3))

s+iϵ...........0.018 + 0.003im
s..............0.022 + 0.0im
s-iϵ...........0.018 - 0.003im


In [13]:
ev = linspace(3mπ+1e-2,3.1,50)
ff = 8π*[second_integral(e^2) for e in ev]
plot(ev, real(ff))
plot!(ev, imag(ff))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
<polyline clip-path="url(#clip3202)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.813 43.4077,375.752 54.5932,375.437 65.7788,374.644 76.9644,373.096 88.1499,370.373 99.3355,365.742 110.521,357.729 121.707,343.089 132.892,316.322 
 144.078,278.195 155.263,237.784 166.449,200.066 177.634,166.694 188.82,138.234 200.006,114.516 211.191,95.1133 222.377,79.3078 233.562,66.4589 244.748,55.9798 
 255.933,47.4711 267.119,40.4508 278.305,34.7375 289.49,30.0247 300.676,26.1442 311.861,22.9625 323.047,20.2182 334.232,18.1968 345.418,16.5195 356.604,15.1473 
 367.789,14.0734 378.975,13.2575 390.16,12.6239 401.346,12.1694 412.531,11.9584 423.717,11.8291 434.903,11.811 446.088,11.9256 457.274,12.0904 468.459,12.3803 
 479.645,12.722 490.83,13.0694 502.016,13.4957 513.202,13.9347 524.387,14.3972 535.573,15.0154 546.758,15.4914 557.944,16.0401 569.129,16.5973 580.315,17.1933 
 
 "/>
<polyline clip-path="url(#clip3202)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,375.813 43.4077,375.813 54.5932,375.813 65.7788,375.813 76.9644,375.813 88.1499,375.813 99.3355,375.813 110.521,375.813 121.707,375.813 132.892,375.813 
 144.078,375.813 155.263,375.813 166.449,375.813 177.634,375.813 188.82,375.813 200.006,375.813 211.191,375.813 222.377,375.813 233.562,375.813 244.748,375.813 
 255.933,375.813 267.119,375.813 278.305,375.813 289.49,375.813 300.676,375.813 311.861,375.813 323.047,375.813 334.232,375.813 345.418,375.813 356.604,375.813 
 367.789,375.813 378.975,375.813 390.16,375.813 401.346,375.813 412.531,375.813 423.717,375.813 434.903,375.813 446.088,375.813 457.274,375.813 468.459,375.813 
 479.645,375.813 490.83,375.813 502.016,375.813 513.202,375.813 524.387,375.813 535.573,375.813 546.758,375.813 557.944,375.813 569.129,375.813 580.315,375.813 
 
 "/>
 
 
 
 
 y1 
 
 
 
 y2

In [14]:
using Interpolations

In [15]:
ρsitp = interpolate(([e^2 for e in ev],), real(ff/(8π)), Gridded(Linear()));
ρsi(s) = ρsitp[s]

ρsi (generic function with 1 method)

### Comparison of the different energy dependent widths
 * two-body phase for $X\to\rho\pi$, space stable $\rho$ (dashed black)
 * non-symmetrized phase space, integral over Dalitz plot with a single $\rho$-band (blue)
 * symmetrized decay amplitude, includes interference (red)

In [16]:
# complex agrument
ρ(s::Complex{Float64}) = 1/(8*π*s)/(2π)*begin
    s_f = (sqrt(s)-mπ)^2 # final piont
    s_m = real(s_f)+1e-5im*imag(s_f); # point in the middle
    integrand = s1->begin
        f1_I(s1)*f1_II(s1) * sqrt(λ(s1,mπ2,mπ2))/(8π*s1) * 
            sqrt(λ(s,s1,mπ2))
    end
    int1 = quadgk(integrand, 4*mπ2, s_m)[1]
    int2 = quadgk(integrand, s_m, s_f)[1]
    int1+int2
end

ρ(s::Float64) = quadgk(s1->begin
        sqrt(λ(s,s1,mπ2))/(8π*s) * 
            real(f1_I(s1)*f1_II(s1)) * sqrt(λ(s1,mπ2,mπ2))/(8π*s1) / (2π)
        end,4mπ2,(sqrt(s)-mπ)^2)[1]

ρ (generic function with 2 methods)

In [84]:
gg = 8π*[ρ(e^2) for e in ev]
plot(ev, real(gg), lab="qtb ph.sp.",leg=:bottomright, xlab = "M3pi", size=(800,500))
plot!(e->8π*ρsi(e^2), 0.5, 3.0, lab="qtb, symm")
plot!(e->sqrt(λ(e^2,mπ2,0.77^2))/e^2, 0.77+mπ, 3.0, lab="two-body", lc=:black, ls=:dash)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 M3pi 
 
<polyline clip-path="url(#clip6002)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 32.2221,460.065 47.4893,460.01 62.7565,459.72 78.0237,458.985 93.2909,457.529 108.558,454.913 123.825,450.32 139.093,442.028 154.36,426.132 169.627,396.416 
 184.894,356.273 200.161,317.973 215.429,285.144 230.696,257.217 245.963,233.234 261.23,212.426 276.497,194.211 291.765,178.146 307.032,163.885 322.299,151.156 
 337.566,139.738 352.833,129.451 368.101,120.146 383.368,111.698 398.635,104.003 413.902,96.9725 429.169,90.5302 444.437,84.6113 459.704,79.1596 474.971,74.1264 
 490.238,69.4693 505.505,65.1512 520.773,61.1396 536.04,57.4057 551.307,53.9241 566.574,50.6723 581.841,47.6303 597.109,44.7802 612.376,42.106 627.643,39.5934 
 642.91,37.2293 658.177,35.0023 673.445,32.9018 688.712,30.9182 703.979,29.043 719.246,27.2684 734.513,25.5871 749.781,23.9927 765.048,22.4792 780.315,21.0413 
 
 "/>
<polyline clip-path="url(#clip6002)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.3609,459.842 56.0691,459.802 57.7773,459.761 59.4855,459.721 75.2805,458.915 91.0756,457.267 106.871,454.29 122.666,449.133 132.158,443.876 141.65,437.011 
 151.142,426.775 160.634,411.045 168.66,395.029 176.685,373.521 184.711,350.655 192.736,327.057 210.172,277.007 227.608,232.299 235.645,214.322 243.682,197.379 
 251.719,182.564 259.756,168.414 268.373,155.676 276.99,143.368 285.607,133.398 294.223,123.839 314.19,106.182 334.156,92.4455 351.9,82.8596 369.644,75.4051 
 385.204,70.1078 400.764,65.7627 434.692,58.7657 468.823,54.498 506.558,51.6936 542.463,50.4175 579.795,50.1159 611.221,50.4136 649.588,51.3093 685.654,52.4051 
 743.436,54.6179 
 "/>
<polyline clip-path="url(#clip6002)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 167.834,434.065 168.548,423.278 169.263,414.99 169.977,407.996 170.691,401.827 171.406,396.246 172.12,391.11 172.835,386.326 176.137,367.334 179.44,351.602 
 182.743,337.896 186.045,325.618 192.651,304.099 199.256,285.491 205.862,269.011 212.467,254.195 219.073,240.735 225.678,228.415 241.557,202.559 257.435,180.871 
 270.86,165.059 284.285,151.123 313.452,125.925 340.341,107.393 369.169,91.2148 402.569,76.05 432.251,65.0224 458.28,56.8305 486.658,49.1634 515.204,42.5363 
 546.766,36.2412 576.797,31.0762 608.021,26.4076 634.306,22.9431 666.396,19.1993 696.562,16.0944 744.89,11.811 
 "/>
 
 
 
 
 qtb ph.sp. 
 
 
 
 qtb, symm 
 
 
 
 two-body

## $a_1(1260)$ parametrization

the amplitude is parametrized by the Breit-Wigner formula
$$
a(s) = \frac{c}{m^2-s-im\Gamma(s)},\text{ where }
\Gamma(s) = \Gamma_0(s)\frac{\rho(s)}{\rho(m^2)} \frac{m}{\sqrt{s}}
$$


In [18]:
const ma1=1.299; const Γa1=0.380; # from COMPASS fit
const ρ0s = second_integral(ma1^2); # precalculation for Γ0

In [19]:
function a1(s,m0,Γ0)
    Γ = Γ0*m0/sqrt(s)*ρ(s)/ρ(m0^2)
    m0^2-s-1im*m0*Γ
end
function a1s(s::Float64,m0,Γ0)
    Γ = Γ0*m0/sqrt(s)*ρsi(s)/ρ0s
    m0^2-s-1im*m0*Γ
end

function a1s(s::Complex{Float64},m0,Γ0)
    Γ = Γ0*m0/sqrt(s)*second_integral(s)/ρ0s
    m0^2-s-1im*m0*Γ
end

a1s (generic function with 2 methods)

In [20]:
println("a1^{-1}..............",round( a1(1.1^2-0.01im, ma1, Γa1),4))
println("a1^{-1}.sym..........",round(a1s(1.1^2-0.01im, ma1, Γa1),4))

a1^{-1}..............0.4738 - 0.3957im
a1^{-1}.sym..........0.4731 - 0.3799im


How the symmetrization changes $a_1(1260)$ line shape

In [54]:
plot(size=(900,500),xlab="M3pi, (GeV)",ylab="a(s): real, imag")
plot!(e->real(1/a1(e^2,ma1,Γa1)), 0.5,3, lab="real")
plot!(e->imag(1/a1(e^2,ma1,Γa1)), 0.5,3, lab="imag")
plot!(e->real(1/a1s(e^2,ma1,Γa1)), 0.5,3, lab="real sym")
plot!(e->imag(1/a1s(e^2,ma1,Γa1)), 0.5,3, lab="imag sym")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 M3pi, (GeV) 
 
 
 a(s): real, imag 
 
<polyline clip-path="url(#clip5102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.0072,209.177 47.0779,208.725 49.1487,208.264 51.2194,207.794 70.3664,202.947 89.5133,197.061 108.66,189.876 127.807,181.044 139.314,174.769 150.82,167.674 
 162.327,159.781 173.833,151.429 183.562,144.675 193.291,138.934 203.019,134.504 212.748,131.334 223.316,129.322 233.884,129.143 239.168,129.968 244.452,131.584 
 249.736,134.165 255.02,137.916 259.891,142.613 264.763,148.703 269.634,156.393 274.506,165.883 279.377,177.347 284.248,190.91 289.12,206.616 293.991,224.403 
 299.214,245.554 304.436,268.435 309.659,292.429 314.882,316.781 320.104,340.668 325.327,363.295 327.938,373.917 330.55,383.982 333.161,393.433 335.772,402.227 
 338.798,411.555 341.823,419.938 344.849,427.371 347.874,433.87 350.9,439.465 353.925,444.199 356.951,448.124 359.976,451.297 366.027,455.635 372.078,457.709 
 378.129,457.99 384.18,456.892 394.935,452.593 405.689,446.502 416.444,439.586 427.199,432.44 446.061,420.307 464.922,409.314 485.487,398.851 506.051,389.886 
 526.738,382.182 547.425,375.587 570.297,369.361 593.168,364.053 614.93,359.698 636.692,355.903 681.947,349.421 720.042,345.089 766.551,340.817 810.272,337.565 
 880.315,333.469 
 "/>
<polyline clip-path="url(#clip5102)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.0779,312.655 51.2194,312.629 89.5133,311.941 127.807,309.213 139.314,307.316 150.82,304.302 162.327,299.36 173.833,291.181 183.562,280.753 193.291,267.118 
 203.019,251.156 212.748,233.572 223.316,212.779 233.884,189.95 244.452,164.719 255.02,136.922 264.763,109.33 274.506,80.8813 279.377,66.9247 284.248,53.6052 
 289.12,41.3526 293.991,30.6444 299.214,21.4394 304.436,15.1405 309.659,12.1972 314.882,12.8721 320.104,17.192 325.327,24.9363 330.55,35.6663 335.772,48.7888 
 347.874,84.7043 359.976,122.339 366.027,140.151 372.078,156.803 378.129,172.125 384.18,186.061 389.557,197.298 394.935,207.502 400.312,216.739 405.689,225.082 
 416.444,239.394 427.199,251.029 436.63,259.441 446.061,266.491 455.491,272.423 464.922,277.435 485.487,285.918 506.051,291.983 526.738,296.435 547.425,299.743 
 593.168,304.515 636.692,307.168 681.947,308.88 720.042,309.835 766.551,310.634 810.272,311.148 880.315,311.684 
 "/>
<polyline clip-path="url(#clip5102)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.0779,208.725 51.2194,207.794 70.3664,202.947 89.5133,197.061 108.66,189.877 127.807,181.041 139.314,174.769 150.82,167.644 162.327,159.657 173.833,151.058 
 183.562,143.466 193.291,136.573 203.019,130.487 212.748,125.716 218.032,123.952 223.316,122.751 228.6,121.88 233.884,121.993 239.168,123.31 244.452,124.975 
 249.736,127.857 255.02,132.484 259.891,138.171 264.763,144.535 269.634,152.905 274.506,163.45 279.377,175.672 284.248,189.493 289.12,205.688 293.991,224.079 
 299.214,245.361 304.436,268.271 309.659,292.377 314.882,316.784 320.104,340.798 325.327,363.642 330.55,384.513 335.772,403.129 338.798,412.777 341.823,421.445 
 344.849,429.117 347.874,435.8 350.9,441.521 353.925,446.568 356.951,450.778 359.976,454.169 366.027,458.756 372.078,461.002 378.129,461.472 384.18,460.38 
 394.935,455.989 405.689,449.632 416.444,442.455 427.199,434.984 446.061,422.336 464.922,410.906 485.487,400.064 506.051,390.81 526.738,382.891 547.425,376.133 
 570.297,369.774 593.168,364.367 614.93,359.943 636.692,356.095 681.947,349.54 720.042,345.17 766.551,340.869 810.272,337.601 880.315,333.489 
 "/>
<polyline clip-path="url(#clip5102)" style="stroke:#c271d2; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.0779,312.64 51.2194,312.617 89.5133,311.911 127.807,309.252 139.314,307.325 150.82,304.564 162.327,300.089 173.833,292.596 183.562,284.004 193.291,271

## The resonance pole position

First, repeat the exercise with the quasi-two-body phase space in the width $\Gamma(s)$

In [63]:
sx1 = 0.5:0.05:3.0
sy1 = -0.8:0.02:0.8
fss = [a1(sr+1im*si,ma1,Γa1) for si in sy1, sr in sx1];

In [64]:
contour(sx1, sy1, [log(abs(f)) for f in fss], levels=40,
    title="Log@Abs@1/A", xlab = "Re(s), GeV^2", ylab = "Im(s), GeV^2", size=(800,500))
annotate!(1.2 ,-0.2,"Rho-Pi cut")
annotate!(1.2 , 0.2,"Rho-Pi cut")
annotate!(1.8 ,-0.65,"a1(1260) pole")
hline!([0],lc=:black,label="",ls=:dash)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 -0.6 
 
 
 -0.3 
 
 
 0.0 
 
 
 0.3 
 
 
 0.6 
 
 
 Log@Abs@1/A 
 
 
 Re(s), GeV^2 
 
 
 Im(s), GeV^2 
 
 
<polyline clip-path="url(#clip6302)" style="stroke:#f3e055; stroke-width:1; stroke-opacity:1; fill:none" points="
 121.368,460.593 119.434,458.631 116.19,455.229 111.258,449.865 106.551,444.502 106.525,444.472 102.068,439.138 97.7933,433.774 93.7237,428.411 93.617,428.266 
 89.8297,423.047 86.1265,417.683 82.6122,412.319 80.7086,409.282 79.2621,406.956 76.0618,401.592 73.0361,396.228 70.1833,390.865 67.8001,386.11 67.4931,385.501 
 64.9052,380.137 62.4773,374.773 60.2075,369.41 58.0937,364.046 56.1336,358.682 54.8916,355.02 
 "/>
<polyline clip-path="url(#clip6302)" style="stroke:#fbad13; stroke-width:1; stroke-opacity:1; fill:none" points="
 155.594,460.593 155.281,455.229 154.977,449.865 154.682,444.502 145.251,440.396 142.313,439.138 142.023,433.774 141.741,428.411 141.468,423.047 141.204,417.683 
 140.949,412.319 140.702,406.956 140.464,401.592 140.234,396.228 140.014,390.865 139.801,385.501 139.598,380.137 139.403,374.773 139.216,369.41 139.038,364.046 
 138.869,358.682 138.709,353.319 138.557,347.955 138.414,342.591 138.28,337.228 138.156,331.864 138.042,326.5 137.94,321.136 137.853,315.773 137.788,310.409 
 137.758,305.045 137.797,299.682 138.008,294.318 138.762,288.954 141.261,283.59 145.251,279.137 147.878,278.227 157.915,272.863 158.159,272.703 167.743,267.499 
 171.068,265.497 176.735,262.136 183.976,257.668 185.398,256.772 193.854,251.408 196.885,249.473 202.217,246.044 209.793,241.2 210.612,240.681 219.158,235.317 
 222.702,233.117 227.982,229.953 235.61,225.461 237.179,224.59 246.991,219.226 248.519,218.401 257.702,213.862 261.427,212.049 269.619,208.498 274.336,206.479 
 283.402,203.135 287.244,201.728 300.152,197.817 300.346,197.771 313.061,194.72 325.969,192.492 326.763,192.407 338.878,191.082 351.786,190.528 364.695,190.837 
 377.603,192.031 379.945,192.407 390.512,194.108 403.42,197.142 405.48,197.771 416.329,201.153 421.459,203.135 429.237,206.234 434.045,208.498 442.146,212.477 
 444.619,213.862 453.772,219.226 455.054,220.01 461.833,224.59 467.963,229.016 469.167,229.953 475.775,235.317 480.871,239.767 481.864,240.681 487.451,246.044 
 492.616,251.408 493.779,252.681 497.419,256.772 501.877,262.136 505.996,267.499 506.688,268.45 509.88,272.863 513.492,278.227 516.828,283.59 519.596,288.41 
 519.913,288.954 522.861,294.318 525.579,299.682 528.076,305.045 530.363,310.409 532.449,315.773 532.505,315.929 534.451,321.136 536.271,326.5 537.915,331.864 
 539.389,337.228 540.698,342.591 541.849,347.955 542.847,353.319 543.696,358.682 544.401,364.046 544.965,369.41 545.391,374.773 545.413,375.187 545.699,380.137 
 545.865,385.501 545.888,390.865 545.767,396.228 545.504,401.592 545.413,402.784 545.116,406.956 544.597,412.319 543.939,417.683 543.138,423.047 542.191,428.411 
 541.094,433.774 539.842,439.138 538.43,444.502 536.852,449.865 535.103,455.229 533.176,460.593 
 "/>
<polyline clip-path="url(#clip6302)" style="stroke:#fab91f; stroke-width:1; stroke-opacity:1; fill:none" points="
 562.619,460.593 564.251,455.229 565.736,449.865 567.078,444.502 568.282,439.138 569.351,433.774 570.289,428.411 571.1,423.047 571.23,422.032 571.818,417.683 
 572.416,412.319 572.886,406.956 573.23,401.592 573.45,396.228 573.546,390.865 573.517,385.501 573.364,380.137 573.087,374.773 572.684,369.41 572.155,364.046 
 571.496,358.682 571.23,356.868 570.736,353.319 569.864,347.955 568.862,342.591 567.725,337.228 566.451,331.864 565.034,326.5 563.47,321.136 561.754,315.773 
 559.881,310.409 558.322,306.288 557.866,305.045 555.757,299.682 553.471,294.318 551.002,288.954 548.342,283.59 545.482,278.227 545.413,278.105 542.516,272.863 
 539.33,267.499 535.911,262.136 532.505,257.138 532.255,256.772 528.428,251.408 524.323,246.044 519.928,240.681 519.596,

In [ ]:
using NLopt

In [94]:
a1_abs(x::Vector, grad::Vector) = abs(a1(x[1]+1im*x[2],ma1,Γa1))

# find minimum which of course suppose to be zero
opt = Opt(:LN_COBYLA, 2)
xtol_rel!(opt,1e-4)

min_objective!(opt, a1_abs)

(minf,a1p,ret) = optimize(opt, [1.6, -0.3])
println("got $minf at $a1p after $count iterations (returned $ret)")

got 0.00016250641997627178 at [1.65805, -0.524488] after count iterations (returned XTOL_REACHED)


In [98]:
(ma1pole,Γa1pole) = 
begin
    sp = a1p[1]+1im*a1p[2]
    rep,imp = reim(sqrt(sp))
    rep,-2.*imp
end
# header
print_head() = begin
    println(rpad("resonance:",20),
            rpad("(mBW, ΓBW)",20),
            " -> ",
            lpad("(m_pole, Γ_pole)",20))
    println(rpad("",63,"-"))
end
print_head()
begin
        println(rpad("a1(1260)",20),
        rpad((ma1,Γa1),20),
        " -> ",
        lpad((round(ma1pole,3),round(Γa1pole,3)),20))
end

resonance:          (mBW, ΓBW)           ->     (m_pole, Γ_pole)
---------------------------------------------------------------
a1(1260)            (1.299, 0.38)        ->       (1.303, 0.402)


#### The second, perform analytical continuation of the symmetrized integral.

In [ ]:
# The next cell takes `10h` to calculate the integrals. Be careful exercuting!
sx = linspace(0.5,3.0,50)
sy = linspace(-0.8,0.8,50)
fss = [begin
        val = a1s(sr+1im*si,ma1,Γa1)
        println(sr+1im*si, " -> ", val)
        val
        end for si in sy, sr in sx]
# write results to file
writedlm(pwd()*"/symm.qtb.re",real(fss))
writedlm(pwd()*"/symm.qtb.im",imag(fss))
writedlm(pwd()*"/symm.qtb.x",collect(sx))
writedlm(pwd()*"/symm.qtb.y",collect(sy))

Read results of the previously performed calculations

In [61]:
rss = readdlm(pwd()*"/symm.qtb.re") + 1im*readdlm(pwd()*"/symm.qtb.im")
sx = vcat(readdlm(pwd()*"/symm.qtb.x")...)
sy = vcat(readdlm(pwd()*"/symm.qtb.y")...);

In [86]:
contour(sx, sy, [log(abs(f)) for f in rss], levels=10,
    title="Log@Abs@1/A", xlab = "Re(s), GeV^2", ylab = "Im(s), GeV^2", size=(800,500))
annotate!(1.2 ,-0.2,"Rho-Pi cut")
annotate!(1.2 , 0.2,"Rho-Pi cut")
annotate!(1.8 ,-0.65,"a1(1260) pole")
hline!([0],lc=:black,label="",ls=:dash)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 -0.6 
 
 
 -0.3 
 
 
 0.0 
 
 
 0.3 
 
 
 0.6 
 
 
 Log@Abs@1/A 
 
 
 Re(s), GeV^2 
 
 
 Im(s), GeV^2 
 
<polyline clip-path="url(#clip2402)" style="stroke:#f7cf3b; stroke-width:1; stroke-opacity:1; fill:none" points="
 119.645,460.593 117.24,451.836 114.961,443.079 112.426,434.322 109.457,425.565 107.579,420.73 106.55,416.807 104.506,408.05 102.311,399.293 100.136,390.536 
 98.0398,381.779 96.065,373.022 94.4073,365.575 94.1538,364.265 92.4429,355.508 90.5177,346.751 88.1414,337.994 85.8244,329.237 83.9003,320.48 82.4477,311.723 
 81.3935,302.965 81.2354,300.197 80.8681,294.208 80.8023,285.451 81.2354,277.56 81.2797,276.694 82.1949,267.937 83.6263,259.18 85.5887,250.423 88.1013,241.666 
 91.1102,232.909 94.4073,224.439 94.5297,224.152 98.3694,215.395 102.176,206.638 105.525,197.881 107.579,191.485 108.333,189.123 110.64,180.366 112.755,171.609 
 114.854,162.852 116.978,154.095 119.358,145.338 120.751,140.195 121.577,136.581 124.151,127.824 127.016,119.067 130.052,110.31 133.266,101.553 133.923,100.042 
 134.196,101.553 135.399,110.31 136.338,119.067 137.24,127.824 138.106,136.581 139.073,145.338 140.105,154.095 141.346,162.852 142.871,171.609 144.979,180.366 
 147.095,186.198 150.747,180.366 155.784,171.609 160.267,163.439 160.614,162.852 165.966,154.095 171.559,145.338 173.439,142.522 177.532,136.581 183.911,127.824 
 186.611,124.413 190.825,119.067 198.196,110.31 199.783,108.52 205.971,101.553 212.954,94.3531 214.5,92.7956 223.591,84.0385 226.126,81.7097 233.439,75.2814 
 239.298,70.4316 244.297,66.5244 252.47,60.4254 256.377,57.7673 265.642,51.7571 270.264,49.0102 278.814,44.0872 286.358,40.2531 291.986,37.4757 305.158,31.948 
 306.411,31.4961 
 "/>
<polyline clip-path="url(#clip2402)" style="stroke:#f7cf3b; stroke-width:1; stroke-opacity:1; fill:none" points="
 472.038,31.4961 476.393,33.166 489.564,39.1576 491.689,40.2531 502.736,46.0535 507.69,49.0102 515.908,54.1663 521.072,57.7673 529.08,63.5689 532.825,66.5244 
 542.252,74.475 543.151,75.2814 552.417,84.0385 555.424,87.0883 560.738,92.7956 568.198,101.553 568.596,102.44 571.615,110.31 570.321,119.067 568.596,122.384 
 566.045,127.824 559.188,136.581 557.212,145.338 560.686,154.095 565.073,162.852 568.596,170.087 569.34,171.609 573.631,180.366 576.641,189.123 579.041,197.881 
 581.768,204.644 582.564,206.638 587.515,215.395 593.471,224.152 594.94,226.4 599.145,232.909 604.359,241.666 608.112,248.549 609.132,250.423 613.578,259.18 
 617.621,267.937 621.284,276.518 621.36,276.694 625.465,285.451 629.794,294.208 633.449,302.965 634.455,306.335 636.212,311.723 638.71,320.48 641.182,329.237 
 643.143,337.994 644.742,346.751 645.976,355.508 646.956,364.265 647.031,373.022 637.405,381.779 634.455,383.96 625.919,390.536 621.284,395.499 618.13,399.293 
 614.347,408.05 610.317,416.807 608.112,421.192 605.907,425.565 601.29,434.322 596.29,443.079 594.94,445.299 590.942,451.836 585.218,460.593 
 "/>
<polyline clip-path="url(#clip2402)" style="stroke:#fa9807; stroke-width:1; stroke-opacity:1; fill:none" points="
 432.872,460.593 436.877,458.17 446.195,451.836 450.049,449.087 457.673,443.079 463.221,438.357 467.606,434.322 476.393,425.748 476.57,425.565 484.563,416.807 
 489.564,410.878 491.864,408.05 498.458,399.293 502.736,393.015 504.402,390.536 509.805,381.779 515.582,373.022 515.908,372.811 529.08,364.907 530.175,364.265 
 542.252,358.91 548.864,355.508 552.109,346.751 548.768,337.994 544.636,329.237 542.252,324.842 539.891,320.48 534.842,311.723 529.866,302.965 529.08,301.796 
 523.922,294.208 516.532,285.451 515.908,284.641 509.667,276.694 503.099,267.937 502.736,267.484 495.818,259.18 489.564,252.469 487.539,250.423 478.096,241.666 
 476.393,240.2 467.118,232.909 463.221,230.045 454.183,224.152 450.049,221.595 438.359,215.395 436.877,214.237 425.621,206.638 423.705,203.634 419.478,197.881 
 

### Finding exact pole position

In [ ]:
@time (minf,a1sp,ret) = begin
    counts = 0
    function a1s_abs(x::Vector, grad::Vector) 
        global counts
        counts::Int += 1
        val = abs(a1s(x[1]+1im*x[2],ma1,Γa1))
        println("f_$count($x) = $val")
        val
    end

    # find minimum which of course suppose to be zero
    opt = Opt(:LN_COBYLA, 2)
    xtol_rel!(opt,1e-4)

    min_objective!(opt, a1s_abs)

    (minf,a1sp,ret) = optimize(opt, [1.66, -0.25])
    println("got $minf at $a1sp after $counts iterations (returned $ret)")
    (minf,a1sp,ret)
end;

In [104]:
println("got $minf at $a1sp after $count iterations (returned $ret)")

got 9.923656919921482e-5 at [1.66053, -0.26938] after 36 iterations (returned XTOL_REACHED)


In [102]:
(ma1spole,Γa1spole) = 
begin
    sp = a1sp[1]+1im*a1sp[2]
    rep,imp = reim(sqrt(sp))
    rep,-2.*imp
end
print_head()
begin
        println(rpad("a1(1260)",20),
        rpad((ma1,Γa1),20),
        " -> ",
        lpad((round(ma1spole,3),round(Γa1spole,3)),20))
end

resonance:          (mBW, ΓBW)           ->     (m_pole, Γ_pole)
---------------------------------------------------------------
a1(1260)            (1.299, 0.38)        ->       (1.293, 0.208)


### How does the amplitude evolve going to the complex plane

In [25]:
fss_q = [a1(sr+1im*si,ma1,Γa1) for si in sy, sr in sx];

Arrays have incorrect length or dimension.


In [26]:
using Interact
using Plots

In [87]:
@manipulate for yv=1:50
    plot(size=(800,500), title="Amplitude(s) when Im(s) = "*string(round(sy[yv],2))*", GeV^2",
        xlab="Re(s), GeV^2", ylab="a(s): real, imag")
    data1 = [1/f for f in fss_q[yv,:]]
    plot!(sx,[real(data1) imag(data1)],lab=["real" "imag"], lc=[:black :red], ls=:dash)
    data2 = [1/f for f in rss[yv,:]]
    plot!(sx,[real(data2) imag(data2)],lab=["real,symm" "imag,symm"], lc=[:black :red])
end

Interact.Options{:SelectionSlider,Any}(73: "input-19" = 25 Any , "yv", 25, "25", 25, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(2, "2"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(39, "39"),Pair{Any,Any}(46, "46"),Pair{Any,Any}(25, "25"),Pair{Any,Any}(42, "42"),Pair{Any,Any}(29, "29"),Pair{Any,Any}(8, "8"),Pair{Any,Any}(20, "20"),Pair{Any,Any}(14, "14")…)), Any[], Any[], true, "horizontal")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 3.0 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 Amplitude(s) when Im(s) = -0.02, GeV^2 
 
 
 Re(s), GeV^2 
 
 
 a(s): real, imag 
 
<polyline clip-path="url(#clip1802)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 45.0072,203.222 60.0135,197.96 75.0197,192.193 90.026,185.862 105.032,178.937 120.039,171.518 135.045,164.048 150.051,157.415 165.057,152.281 180.064,148.559 
 195.07,145.928 210.076,144.247 225.083,143.587 240.089,144.191 255.095,146.438 270.101,150.822 285.108,157.925 300.114,168.365 315.12,182.705 330.127,201.32 
 345.133,224.229 360.139,250.95 375.145,280.43 390.152,311.13 405.158,341.279 420.164,369.205 435.17,393.63 450.177,413.833 465.183,429.636 480.189,441.285 
 495.196,449.281 510.202,454.237 525.208,456.767 540.214,457.427 555.221,456.69 570.227,454.94 585.233,452.478 600.24,449.536 615.246,446.287 630.252,442.863 
 645.258,439.358 660.265,435.841 675.271,432.36 690.277,428.949 705.284,425.631 720.29,422.421 735.296,419.327 750.302,416.353 765.309,413.501 780.315,410.769 
 
 "/>
<polyline clip-path="url(#clip1802)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" stroke-dasharray="8, 5" points="
 45.0072,318.454 60.0135,317.535 75.0197,316.13 90.026,313.969 105.032,310.603 120.039,305.336 135.045,297.342 150.051,286.333 165.057,273.109 180.064,258.702 
 195.07,243.572 210.076,227.739 225.083,211.061 240.089,193.379 255.095,174.598 270.101,154.736 285.108,133.985 300.114,112.772 315.12,91.8121 330.127,72.1246 
 345.133,54.9825 360.139,41.7567 375.145,33.6667 390.152,31.4961 405.158,35.3815 420.164,44.7714 435.17,58.5743 450.177,75.4253 465.183,93.9598 480.189,113.009 
 495.196,131.692 510.202,149.419 525.208,165.849 540.214,180.83 555.221,194.335 570.227,206.42 585.233,217.182 600.24,226.74 615.246,235.221 630.252,242.745 
 645.258,249.425 660.265,255.364 675.271,260.653 690.277,265.373 705.284,269.595 720.29,273.38 735.296,276.781 750.302,279.844 765.309,282.61 780.315,285.114 
 
 "/>
<polyline clip-path="url(#clip1802)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.0072,203.232 60.0135,197.981 75.0197,192.232 90.026,185.924 105.032,179.015 120.039,171.542 135.045,163.774 150.051,156.322 165.057,149.823 180.064,144.555 
 195.07,140.6 210.076,138.04 225.083,137.034 240.089,137.801 255.095,140.652 270.101,145.971 285.108,154.208 300.114,165.823 315.12,181.214 330.127,200.654 
 345.133,224.072 360.139,251 375.145,280.471 390.152,311.079 405.158,341.199 420.164,369.256 435.17,394.001 450.177,414.657 465.183,431.018 480.189,443.214 
 495.196,451.672 510.202,456.971 525.208,459.765 540.214,460.593 555.221,459.935 570.227,458.193 585.233,455.681 600.24,452.647 615.246,449.286 630.252,445.73 
 645.258,442.081 660.265,438.415 675.271,434.789 690.277,431.231 705.284,427.772 720.29,424.429 735.296,421.209 750.302,418.113 765.309,415.148 780.315,412.313 
 
 "/>
<polyline clip-path="url(#clip1802)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.0072,318.454 60.0135,317.575 75.0197,316.263 90.026,314.297 105.032,311.316 120.039,306.756 135.045,299.897 150.051,290.306 165.057,278.225 180.064,264.249 
 195.07,248.804 210.076,232.101 225.083,214.232 240.089,195.27 255.095,175.298 270.101,154.473 285.108,133.081 300.114,111.585 315.12,90.6714 330.127,71.2797 
 345.133,54.5416 360.139,41.6738 375.145,33.7873 390.152,31.5661 405.158,35.2038 420.164,44.23 435.17,57.6638 450.177,74.263 465.183,92.6697 480.189,111.765 
 495.196,130.641 510.202,148.655 525.208,165.415 540.214,180.741 555.221,194.583 570.227,206.979 585.233,218.018 600.24,227.815 615.246,236.499 630.252,244.19 
 645.258,251.002 660.265,257.045 675.271,262.416 690.277,267.192 705.284,271.452 720.29,275.263 735.296,278.677 750.302,281.74 765.309,284.498 780.31